In [318]:
# clear variable cache
%reset -f

In [319]:
# Data
import pandas as pd
import numpy as np
import datetime

# Notebook Settings
import os

In [320]:
# set wd
os.getcwd()
os.chdir('/home/ad-frazier/data_science/MSBA_320/final_project/clean')

# set pandas options
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

## Import GDP Data -- Dataset Core

In [321]:
df_gdp = pd.read_csv('gdp_state_annual_clean.csv')

In [322]:
df_gdp.head(2)

,state,year,subsidies,taxes_on_production_and_imports_(topi),gdp_compensation,gdp_current_dollar,gdp_gross_operating_surplus
0,AK,1997,-62900000.00,2464700000.00,12347800000.00,25810800000.00,11061300000.00
1,AK,1998,-54700000.00,2003900000.00,12889600000.00,24227500000.00,9388700000.00


In [323]:
# Get unique states for join integrity confirmation
print('unique states:', df_gdp.state.unique().shape[0])
print('earliest year:', df_gdp.year.min())
print('earliest year:', df_gdp.year.max())

unique states: 51
earliest year: 1997
earliest year: 2020


In [324]:
df_gdp.shape

(1224, 7)

## Import Personal Income + Population

In [325]:
per_income = pd.read_csv('per_capita_personal_income-population_state_annual_clean.csv')

In [326]:
per_income.head(2)

,year,state,per_capita_personal_income,population
0,1929,AL,319.00,2644000.00
1,1929,AR,303.00,1852000.00


# Join 1: GDP + Personal Income and Population

In [327]:
df_core = df_gdp.merge(per_income, how='inner', on=['state','year'])

df_core.head(2)

,state,year,subsidies,taxes_on_production_and_imports_(topi),gdp_compensation,gdp_current_dollar,gdp_gross_operating_surplus,per_capita_personal_income,population
0,AL,1997,-274500000.00,6754800000.00,61083800000.00,104811900000.00,37247900000.00,21516.00,4367935.00
1,AL,1998,-292900000.00,6968300000.00,64168600000.00,110212000000.00,39368100000.00,22668.00,4404701.00


In [328]:
mov_col = df_core.pop('population')

df_core.insert(2, 'population', mov_col)

df_core.head(3)

,state,year,population,subsidies,taxes_on_production_and_imports_(topi),gdp_compensation,gdp_current_dollar,gdp_gross_operating_surplus,per_capita_personal_income
0,AL,1997,4367935.00,-274500000.00,6754800000.00,61083800000.00,104811900000.00,37247900000.00,21516.00
1,AL,1998,4404701.00,-292900000.00,6968300000.00,64168600000.00,110212000000.00,39368100000.00,22668.00
2,AL,1999,4430141.00,-388400000.00,7329600000.00,67225100000.00,115680100000.00,41513700000.00,23333.00


In [329]:
df_core['per_cap_gdp_current_dollar'] = df_core['gdp_current_dollar']/df_core['population']

df_core.head(2)

,state,year,population,subsidies,taxes_on_production_and_imports_(topi),gdp_compensation,gdp_current_dollar,gdp_gross_operating_surplus,per_capita_personal_income,per_cap_gdp_current_dollar
0,AL,1997,4367935.00,-274500000.00,6754800000.00,61083800000.00,104811900000.00,37247900000.00,21516.00,23995.76
1,AL,1998,4404701.00,-292900000.00,6968300000.00,64168600000.00,110212000000.00,39368100000.00,22668.00,25021.45


In [330]:
# convert remaing gdp measures to per capita
df_core[['subsidies','taxes_on_production_and_imports_(topi)','gdp_compensation','gdp_gross_operating_surplus']] = df_core[['subsidies','taxes_on_production_and_imports_(topi)','gdp_compensation','gdp_gross_operating_surplus']].div(df_core['population'], axis=0)

df_core.head(2)

,state,year,population,subsidies,taxes_on_production_and_imports_(topi),gdp_compensation,gdp_current_dollar,gdp_gross_operating_surplus,per_capita_personal_income,per_cap_gdp_current_dollar
0,AL,1997,4367935.00,-62.84,1546.45,13984.59,104811900000.00,8527.58,21516.00,23995.76
1,AL,1998,4404701.00,-66.50,1582.01,14568.21,110212000000.00,8937.75,22668.00,25021.45


## Import Poverty Rate

In [331]:
poverty_rate = pd.read_csv('poverty_rate_state_annual_clean.csv')

In [332]:
poverty_rate.head(2)

,state,poverty_rate,year
0,AL,0.15,2020
1,AK,0.13,2020


## JOIN 2: df_core + poverty rate

In [333]:
def magic_merge(df1, df2):
    df1 = df1.merge(df2, how='left', on=['state','year'])
    return df1

In [334]:
df_core = magic_merge(df_core, poverty_rate)

df_core.head(2)

,state,year,population,subsidies,taxes_on_production_and_imports_(topi),gdp_compensation,gdp_current_dollar,gdp_gross_operating_surplus,per_capita_personal_income,per_cap_gdp_current_dollar,poverty_rate
0,AL,1997,4367935.00,-62.84,1546.45,13984.59,104811900000.00,8527.58,21516.00,23995.76,0.16
1,AL,1998,4404701.00,-66.50,1582.01,14568.21,110212000000.00,8937.75,22668.00,25021.45,0.14


## Import gini_index

In [335]:
gini_index = pd.read_csv('gini_index_state_annual_clean.csv')

gini_index.head(2)

,state,year,gini_index
0,AL,2006,0.47
1,AL,2007,0.47


In [336]:
df_core.head(2)

,state,year,population,subsidies,taxes_on_production_and_imports_(topi),gdp_compensation,gdp_current_dollar,gdp_gross_operating_surplus,per_capita_personal_income,per_cap_gdp_current_dollar,poverty_rate
0,AL,1997,4367935.00,-62.84,1546.45,13984.59,104811900000.00,8527.58,21516.00,23995.76,0.16
1,AL,1998,4404701.00,-66.50,1582.01,14568.21,110212000000.00,8937.75,22668.00,25021.45,0.14


In [337]:
df_core = magic_merge(df_core, gini_index)

df_core.head(2)

,state,year,population,subsidies,taxes_on_production_and_imports_(topi),gdp_compensation,gdp_current_dollar,gdp_gross_operating_surplus,per_capita_personal_income,per_cap_gdp_current_dollar,poverty_rate,gini_index
0,AL,1997,4367935.00,-62.84,1546.45,13984.59,104811900000.00,8527.58,21516.00,23995.76,0.16,NaN
1,AL,1998,4404701.00,-66.50,1582.01,14568.21,110212000000.00,8937.75,22668.00,25021.45,0.14,NaN


## Import Median Age

In [338]:
median_age = pd.read_csv('median_age_state_annual_clean.csv')

median_age.head(2)

,state,year,median_age
0,AL,2005,37.40
1,AL,2006,37.20


In [339]:
df_core = magic_merge(df_core, median_age)

df_core.sample(5)

,state,year,population,subsidies,taxes_on_production_and_imports_(topi),gdp_compensation,gdp_current_dollar,gdp_gross_operating_surplus,per_capita_personal_income,per_cap_gdp_current_dollar,poverty_rate,gini_index,median_age
771,NY,2000,19001780.00,-204.58,2992.06,25200.13,841181300000.00,16280.95,36090.00,44268.55,0.14,NaN,NaN
896,RI,2005,1067916.00,-158.53,3474.34,23231.79,44374100000.00,15004.46,36984.00,41552.05,0.12,NaN,38.40
1131,WV,2000,1807021.00,-52.63,1968.60,13560.05,42495600000.00,8040.86,22317.00,23516.94,0.15,NaN,NaN
154,DC,2007,574404.00,-1089.65,6391.67,112629.79,93785400000.00,45342.48,59824.00,163274.28,0.18,0.54,34.90
795,OH,2000,11363543.00,-117.76,2218.75,20086.65,392277200000.00,12333.04,28598.00,34520.68,0.10,NaN,NaN


## Import Median Income

In [340]:
median_income = pd.read_csv('median_income_state_annual_clean.csv')

median_income.head(2)

,state,year,median_income
0,AL,2005,36879.00
1,AL,2006,38783.00


In [341]:
df_core = magic_merge(df_core, median_income)

### Import Mean hours worked

In [342]:
mean_hrs = pd.read_csv('hrs_worked_state_annual_clean.csv')

mean_hrs.head(5)

,state,year,mean_usual_hours
0,AL,2005,39.80
1,AL,2006,39.80
2,AL,2007,39.80
3,AL,2008,39.40
4,AL,2009,38.70


In [343]:
df_core = magic_merge(df_core, mean_hrs)

df_core.sample(15)

,state,year,population,subsidies,taxes_on_production_and_imports_(topi),gdp_compensation,gdp_current_dollar,gdp_gross_operating_surplus,per_capita_personal_income,per_cap_gdp_current_dollar,poverty_rate,gini_index,median_age,median_income,mean_usual_hours
490,MI,2007,10001284.00,-96.44,3033.60,23419.45,409615400000.00,14599.66,34929.00,40956.28,0.11,0.45,37.60,47950.00,38.20
915,SC,2000,4024223.00,-83.07,2047.28,16692.27,115320600000.00,10000.12,25133.00,28656.61,0.11,NaN,NaN,NaN,NaN
74,CA,1999,33499204.00,-150.48,2329.44,20330.89,1241899700000.00,14562.67,30773.00,37072.51,0.14,NaN,NaN,NaN,NaN
1038,VA,2003,7366977.00,-126.58,2780.84,24540.93,311665700000.00,15110.57,36188.00,42305.78,0.10,NaN,NaN,NaN,NaN
331,IN,2016,6658366.00,-188.35,2931.98,26570.87,340500700000.00,21824.27,43645.00,51138.78,0.12,0.45,37.60,52314.00,38.70
183,DE,2012,916572.00,-239.48,3391.44,29941.13,62354600000.00,34937.14,44070.00,68030.23,0.14,0.44,39.20,58415.00,37.90
644,ND,2017,766673.00,-1552.42,6286.64,36022.14,55228000000.00,31279.57,52455.00,72035.93,0.12,0.45,35.40,61843.00,40.10
1082,WA,1999,5842564.00,-149.39,3363.11,21510.95,229797900000.00,14607.01,30958.00,39331.69,0.10,NaN,NaN,NaN,NaN
48,AZ,1997,4736990.00,-69.83,1891.24,14751.33,132281000000.00,11352.36,22804.00,27925.12,0.17,NaN,NaN,NaN,NaN
907,RI,2016,1082251.00,-175.19,4483.62,29989.53,57353500000.00,18696.68,49184.00,52994.64,0.11,0.48,40.20,60596.00,37.70


## Import Median Monthly house cost

In [344]:
monthly_house_cost = pd.read_csv('median_monthly_house_cost_state_annual_clean.csv')

monthly_house_cost.head(5)

,state,year,median_monthly_house_cost
0,AL,2005,624.00
1,AL,2006,685.00
2,AL,2007,713.00
3,AL,2008,742.00
4,AL,2009,751.00


In [345]:
df_core = magic_merge(df_core, monthly_house_cost)

df_core.sample(5)

,state,year,population,subsidies,taxes_on_production_and_imports_(topi),gdp_compensation,gdp_current_dollar,gdp_gross_operating_surplus,per_capita_personal_income,per_cap_gdp_current_dollar,poverty_rate,gini_index,median_age,median_income,mean_usual_hours,median_monthly_house_cost
45,AR,2018,2997271.00,-269.31,3370.37,23687.05,127306500000.00,15686.04,43661.00,42474.14,0.16,0.48,38.10,47062.00,39.60,682.00
143,CT,2020,3600260.00,-2355.94,6013.84,40794.22,276422900000.00,32326.50,77664.00,76778.59,0.11,NaN,NaN,NaN,NaN,NaN
572,MS,2017,2983393.00,-176.11,3124.23,20216.01,109962500000.00,13694.07,36907.00,36858.20,0.18,0.48,37.50,43529.00,39.40,595.00
1068,VT,2009,624817.00,-192.22,3874.73,23746.95,26099200000.00,14341.32,40479.00,41770.95,0.09,0.43,41.50,51618.00,37.40,1183.00
496,MI,2013,9947308.00,-120.79,3061.35,24322.30,437274400000.00,16696.20,39566.00,43959.07,0.13,0.46,39.60,48273.00,37.70,869.00


### Import House Values

In [346]:
house_value = pd.read_csv('house_value_quartiles_state_annual_clean.csv')

house_value.head(5)

,state,year,house_low_value_quartile,house_median_value,house_upper_value_quartile
0,AL,2005,59400.00,97500.00,162500.00
1,AL,2006,64200.00,107000.00,174900.00
2,AL,2007,68900.00,115600.00,187700.00
3,AL,2008,68300.00,121500.00,200600.00
4,AL,2009,67400.00,119600.00,197200.00


In [347]:
df_core = magic_merge(df_core, house_value)

df_core.sample(10)

,state,year,population,subsidies,taxes_on_production_and_imports_(topi),gdp_compensation,gdp_current_dollar,gdp_gross_operating_surplus,per_capita_personal_income,per_cap_gdp_current_dollar,poverty_rate,gini_index,median_age,median_income,mean_usual_hours,median_monthly_house_cost,house_low_value_quartile,house_median_value,house_upper_value_quartile
450,MD,2015,6049289.00,-175.61,3831.69,33458.38,369728100000.00,24004.79,55825.00,61119.27,0.10,0.45,38.30,75847.00,39.00,1580.00,200500.00,299800.00,445400.00
110,CO,2011,5119188.00,-194.76,3579.96,27945.84,267516100000.00,20926.48,43658.00,52257.53,0.13,0.46,36.20,55387.00,38.60,1306.00,NaN,NaN,NaN
181,DE,2010,899762.00,-263.07,3277.31,28351.39,58163500000.00,33277.58,40942.00,64643.21,0.12,0.44,38.80,55847.00,38.20,1242.00,NaN,NaN,NaN
309,IL,2018,12888646.00,-266.05,5252.63,36665.69,867535500000.00,25657.78,56512.00,67310.06,0.10,0.49,38.30,65030.00,38.60,1205.00,119300.00,203400.00,326600.00
258,IA,2015,3136111.00,-543.16,3533.17,27884.89,180299300000.00,26616.47,46104.00,57491.36,0.10,0.44,38.10,54736.00,38.80,834.00,84900.00,136100.00,208800.00
1019,UT,2008,2663029.00,-97.48,2542.59,23674.17,117178900000.00,17882.87,33941.00,44002.11,0.08,0.41,28.70,56633.00,37.60,1196.00,168700.00,236000.00,342300.00
994,TX,2007,23831983.00,-124.19,3931.33,24210.56,1191093100000.00,21961.06,37085.00,49978.77,0.17,0.47,33.20,47548.00,40.70,986.00,75600.00,120900.00,186500.00
168,DE,1997,751487.00,-124.29,2100.24,20725.57,33353400000.00,21681.68,27069.00,44383.20,0.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1091,WA,2008,6562231.00,-167.66,4483.23,28649.94,354794100000.00,21100.55,44241.00,54066.08,0.10,0.44,37.30,58078.00,38.50,1428.00,206500.00,308100.00,460700.00
290,IL,1999,12359020.00,-247.21,2623.10,21722.51,460331100000.00,13148.17,31068.00,37246.57,0.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Rent Quartiles

In [348]:
rent_quarts = pd.read_csv('rent_quartiles_state_annual_clean.csv')

rent_quarts.head(5)

,state,year,rent_low_value_quartile,rent_median_value,rent_upper_value_quartile
0,AL,2005,272.00,394.00,541.00
1,AL,2006,282.00,410.00,562.00
2,AL,2007,291.00,427.00,591.00
3,AL,2008,314.00,447.00,619.00
4,AL,2009,322.00,468.00,637.00


In [349]:
df_core = magic_merge(df_core, rent_quarts)

df_core.sample(10)

,state,year,population,subsidies,taxes_on_production_and_imports_(topi),gdp_compensation,gdp_current_dollar,gdp_gross_operating_surplus,per_capita_personal_income,per_cap_gdp_current_dollar,poverty_rate,gini_index,median_age,median_income,mean_usual_hours,median_monthly_house_cost,house_low_value_quartile,house_median_value,house_upper_value_quartile,rent_low_value_quartile,rent_median_value,rent_upper_value_quartile
1093,WA,2010,6743597.00,-202.68,4279.36,27758.87,365497800000.00,22363.69,42348.00,54199.24,0.12,0.44,37.20,55631.00,37.80,1394.00,NaN,NaN,NaN,NaN,NaN,NaN
331,IN,2016,6658366.00,-188.35,2931.98,26570.87,340500700000.00,21824.27,43645.00,51138.78,0.12,0.45,37.60,52314.00,38.70,818.00,85200.00,134800.00,205400.00,477.00,619.00,780.00
76,CA,2001,34479458.00,-365.06,2519.12,22818.77,1375761300000.00,14928.07,34046.00,39900.90,0.13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
832,OK,2013,3848112.00,-190.77,3081.56,23406.65,182694700000.00,21179.01,44017.00,47476.45,0.21,0.46,36.20,45690.00,39.80,737.00,68900.00,116500.00,179700.00,412.00,546.00,717.00
764,NV,2017,2955631.00,-111.41,5027.42,28550.52,160785000000.00,20932.99,47542.00,54399.55,0.13,0.46,38.00,58003.00,38.90,1139.00,178100.00,258200.00,366900.00,714.00,903.00,1175.00
164,DC,2017,680945.00,-1158.83,7686.96,134652.28,133391200000.00,54710.88,79967.00,195891.30,0.14,0.53,34.00,82372.00,40.50,1977.00,382600.00,607200.00,908900.00,920.00,1408.00,2044.00
670,NE,2019,1954927.00,-766.78,4404.21,33546.37,131352400000.00,30006.64,53419.00,67190.44,0.09,0.44,36.80,63229.00,39.40,1020.00,110100.00,172700.00,262800.00,526.00,714.00,926.00
1127,WI,2020,5892323.00,-2132.22,4062.98,33531.43,337714000000.00,21852.06,55030.00,57314.24,0.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
348,KS,2009,2832704.00,-349.14,3117.45,24226.08,125348900000.00,17256.23,39431.00,44250.62,0.14,0.44,35.90,47817.00,38.90,883.00,72100.00,125500.00,195300.00,380.00,515.00,697.00
125,CT,2002,3458749.00,-116.40,3508.61,28041.18,173248300000.00,18656.46,44697.00,50089.87,0.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Import Occupant Ratio

In [350]:
house_occ_ratio = pd.read_csv('household_occupants_ratio_state_annual_clean.csv')

house_occ_ratio.head(5)

,state,year,household_renter_occ_ratio,household_owner_occ_ratio
0,AL,2005,0.28,0.72
1,AL,2006,0.26,0.74
2,AL,2007,0.27,0.73
3,AL,2008,0.27,0.73
4,AL,2009,0.29,0.71


In [351]:
df_core = magic_merge(df_core, house_occ_ratio)

df_core.sample(10)

,state,year,population,subsidies,taxes_on_production_and_imports_(topi),gdp_compensation,gdp_current_dollar,gdp_gross_operating_surplus,per_capita_personal_income,per_cap_gdp_current_dollar,poverty_rate,gini_index,median_age,median_income,mean_usual_hours,median_monthly_house_cost,house_low_value_quartile,house_median_value,house_upper_value_quartile,rent_low_value_quartile,rent_median_value,rent_upper_value_quartile,household_renter_occ_ratio,household_owner_occ_ratio
735,NM,2012,2090254.00,-133.96,2888.26,21116.72,87925200000.00,18193.34,35749.00,42064.36,0.20,0.47,36.80,42558.00,38.40,760.00,89600.00,157500.00,244000.00,466.00,633.00,845.00,0.31,0.69
997,TX,2010,25238863.00,-118.40,3737.30,24141.96,1245959400000.00,21605.83,39034.00,49366.70,0.18,0.47,33.60,48615.00,39.60,1016.00,NaN,NaN,NaN,NaN,NaN,NaN,0.34,0.66
202,FL,2007,18367842.00,-121.68,4180.90,22732.58,782712500000.00,15821.40,39594.00,42613.20,0.12,0.47,40.00,47804.00,39.90,1142.00,148600.00,230400.00,367900.00,601.00,794.00,1036.00,0.29,0.71
187,DE,2016,953030.00,-204.93,4111.83,32021.34,69355000000.00,36844.80,48734.00,72773.16,0.12,0.45,40.60,61757.00,38.70,1095.00,167300.00,243400.00,345900.00,687.00,893.00,1152.00,0.29,0.71
459,ME,2000,1277072.00,-66.09,2590.54,16733.04,36890300000.00,9629.06,27491.00,28886.63,0.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115,CO,2016,5529161.00,-197.32,3757.42,32941.69,329911700000.00,23165.81,52390.00,59667.59,0.09,0.46,36.70,65685.00,39.00,1292.00,210000.00,314200.00,455400.00,762.00,1057.00,1419.00,0.34,0.66
517,MN,2010,5312113.00,-236.25,3540.70,28825.55,274231300000.00,19493.77,42724.00,51623.77,0.11,0.44,37.50,55459.00,37.60,1180.00,NaN,NaN,NaN,NaN,NaN,NaN,0.24,0.76
627,ND,2000,642023.00,-1746.20,2507.85,15879.49,18100100000.00,11551.14,25892.00,28192.29,0.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
574,MS,2019,2968981.00,-273.73,3350.44,21514.22,114733800000.00,14053.24,39184.00,38644.17,0.19,0.49,38.30,45792.00,39.30,648.00,71500.00,128200.00,208200.00,401.00,598.00,792.00,0.31,0.69
672,NH,1997,1189425.00,-62.97,2296.15,17008.81,38206600000.00,12879.92,27975.00,32121.91,0.09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Household Occupied Binary

In [352]:
house_occ_bool = pd.read_csv('household_occupancy_binary_state_annual_clean.csv')

house_occ_bool.head(5)

,state,year,households_occupied_ratio
0,AK,2005,0.85
1,AK,2006,0.83
2,AK,2007,0.84
3,AK,2008,0.84
4,AK,2009,0.83


In [353]:
df_core = magic_merge(df_core, house_occ_bool)

df_core.sample(10)

,state,year,population,subsidies,taxes_on_production_and_imports_(topi),gdp_compensation,gdp_current_dollar,gdp_gross_operating_surplus,per_capita_personal_income,per_cap_gdp_current_dollar,poverty_rate,gini_index,median_age,median_income,mean_usual_hours,median_monthly_house_cost,house_low_value_quartile,house_median_value,house_upper_value_quartile,rent_low_value_quartile,rent_median_value,rent_upper_value_quartile,household_renter_occ_ratio,household_owner_occ_ratio,households_occupied_ratio
227,GA,2008,9504843.00,-122.31,2866.11,24086.89,416377600000.00,16976.20,35438.00,43806.89,0.15,0.47,35.00,50861.00,39.90,1106.00,108900.00,169100.00,268700.00,428.00,630.00,826.00,0.31,0.69,0.86
653,NE,2002,1728292.00,-385.35,2456.53,20626.20,61417300000.00,12839.15,30599.00,35536.41,0.11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
180,DE,2009,891730.00,-200.40,3077.50,28256.65,57849700000.00,33739.81,40856.00,64873.56,0.12,0.43,38.20,56860.00,38.20,1234.00,171400.00,249400.00,348600.00,580.00,778.00,948.00,0.26,0.74,0.83
354,KS,2015,2922671.00,-352.38,3649.74,28423.21,154958200000.00,21298.81,47332.00,53019.38,0.14,0.46,36.20,53906.00,39.10,893.00,81400.00,141200.00,222200.00,434.00,601.00,806.00,0.31,0.69,0.89
1145,WV,2014,1853014.00,-77.55,3183.25,20415.12,72648200000.00,15684.56,36570.00,39205.42,0.21,0.45,41.90,41059.00,38.80,495.00,57800.00,103900.00,171500.00,351.00,485.00,629.00,0.26,0.74,0.83
809,OH,2014,11648446.00,-135.28,3471.64,27317.31,595640900000.00,20481.13,42829.00,51134.80,0.16,0.46,39.40,49308.00,38.10,907.00,81800.00,129100.00,198000.00,443.00,579.00,744.00,0.32,0.68,0.89
439,MD,2004,5546935.00,-146.21,2762.10,25500.50,245124500000.00,16074.59,41641.00,44190.98,0.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
603,NC,2000,8081614.00,-110.73,2178.93,19256.18,278008500000.00,13075.75,27510.00,34400.12,0.12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104,CO,2005,4631888.00,-207.37,2994.31,26058.27,220783700000.00,18820.81,37797.00,47666.03,0.11,NaN,34.70,50652.00,39.60,1220.00,159900.00,223300.00,330000.00,506.00,665.00,890.00,0.29,0.71,0.89
680,NH,2005,1298492.00,-129.23,3241.84,24596.61,56952600000.00,16151.35,40619.00,43860.57,0.06,NaN,39.50,56768.00,39.10,1253.00,166900.00,240100.00,348800.00,559.00,757.00,962.00,0.23,0.77,0.85


### Import Student Funding(revenue)


In [354]:
stud_rev = pd.read_csv('tot_revenue_student_state_annual_clean.csv')

stud_rev.head(5)

,state,year,per_pupil_revenue
0,AL,2018,11415
1,AK,2018,19553
2,AZ,2018,10396
3,AR,2018,11733
4,CA,2018,16014


In [355]:
df_core = magic_merge(df_core, stud_rev)

df_core.sample(10)

,state,year,population,subsidies,taxes_on_production_and_imports_(topi),gdp_compensation,gdp_current_dollar,gdp_gross_operating_surplus,per_capita_personal_income,per_cap_gdp_current_dollar,poverty_rate,gini_index,median_age,median_income,mean_usual_hours,median_monthly_house_cost,house_low_value_quartile,house_median_value,house_upper_value_quartile,rent_low_value_quartile,rent_median_value,rent_upper_value_quartile,household_renter_occ_ratio,household_owner_occ_ratio,households_occupied_ratio,per_pupil_revenue
93,CA,2018,39535906.00,-196.44,4812.13,38737.02,2895101000000.00,29874.42,61509.00,73227.13,0.12,0.49,36.70,75277.00,38.30,1785.00,341100.00,546800.00,852300.00,964.00,1399.00,1953.00,0.44,0.56,0.92,16014.00
1096,WA,2013,6974173.00,-192.27,4801.21,30759.66,419092400000.00,24723.45,47857.00,60092.06,0.12,0.46,37.40,58405.00,38.10,1302.00,164600.00,250800.00,383500.00,647.00,867.00,1196.00,0.36,0.64,0.90,12213.00
468,ME,2009,1329590.00,-130.04,3221.67,21738.05,51123800000.00,13621.19,37157.00,38450.80,0.11,0.43,42.20,45734.00,37.80,952.00,109700.00,177500.00,267700.00,442.00,626.00,822.00,0.24,0.76,0.77,13950.00
301,IL,2010,12845511.00,-223.21,3736.58,28216.93,666404800000.00,20148.11,42278.00,51878.42,0.14,0.47,36.60,52972.00,37.90,1254.00,NaN,NaN,NaN,NaN,NaN,NaN,0.30,0.70,0.90,13818.00
806,OH,2011,11557958.00,-160.31,2974.07,24652.44,528189900000.00,18233.05,39245.00,45699.24,0.15,0.46,39.10,45749.00,37.90,955.00,NaN,NaN,NaN,NaN,NaN,NaN,0.31,0.69,0.88,13153.00
817,OK,1998,3405194.00,-115.03,1649.16,13662.07,81196900000.00,8648.82,22006.00,23845.01,0.14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5811.00
944,SD,2005,775493.00,-1139.66,2980.43,18948.85,30997400000.00,19181.60,33788.00,39971.22,0.12,NaN,37.00,40310.00,40.10,658.00,57300.00,101700.00,156700.00,258.00,409.00,558.00,0.28,0.72,0.89,8967.00
503,MI,2020,10067664.00,-2142.23,3580.01,29660.23,515119900000.00,20067.78,52724.00,51165.78,0.11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,AL,2003,4503491.00,-103.85,1872.92,17116.72,134152600000.00,10902.79,26673.00,29788.58,0.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7349.00
890,RI,1999,1040402.00,-94.58,2480.96,17713.15,32267200000.00,10914.72,28496.00,31014.17,0.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9256.00


### Import per student spending

In [356]:
stud_spend = pd.read_csv('tot_spend_student_state_annual_clean.csv')

stud_spend.head(5)

,state,year,per_pupil_expenditure
0,AL,2018,10107
1,AK,2018,18393
2,AZ,2018,8773
3,AR,2018,10412
4,CA,2018,13831


In [357]:
df_core = magic_merge(df_core, stud_spend)

df_core.sample(15)

,state,year,population,subsidies,taxes_on_production_and_imports_(topi),gdp_compensation,gdp_current_dollar,gdp_gross_operating_surplus,per_capita_personal_income,per_cap_gdp_current_dollar,poverty_rate,gini_index,median_age,median_income,mean_usual_hours,median_monthly_house_cost,house_low_value_quartile,house_median_value,house_upper_value_quartile,rent_low_value_quartile,rent_median_value,rent_upper_value_quartile,household_renter_occ_ratio,household_owner_occ_ratio,households_occupied_ratio,per_pupil_revenue,per_pupil_expenditure
832,OK,2013,3848112.00,-190.77,3081.56,23406.65,182694700000.00,21179.01,44017.00,47476.45,0.21,0.46,36.20,45690.00,39.80,737.00,68900.00,116500.00,179700.00,412.00,546.00,717.00,0.33,0.67,0.86,8918.00,7995.00
433,MD,1998,5204464.00,-109.75,2072.43,19050.18,169982100000.00,11647.96,31483.00,32660.83,0.07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8086.00,7326.00
548,MO,2017,6113669.00,-189.48,2987.89,28889.89,308722100000.00,18808.72,45376.00,50497.03,0.11,0.46,38.50,53578.00,38.80,872.00,93800.00,156700.00,246300.00,465.00,628.00,827.00,0.30,0.70,0.85,12899.00,11034.00
416,MA,2005,6403290.00,-201.35,2948.39,31095.95,341993200000.00,19565.97,44371.00,53408.98,0.10,NaN,38.20,57184.00,38.80,1426.00,244400.00,361500.00,493000.00,509.00,799.00,1122.00,0.31,0.69,0.91,14343.00,12087.00
1158,WY,2003,503453.00,-149.37,4467.94,19771.06,21193600000.00,18006.84,33686.00,42096.48,0.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11107.00,9308.00
431,MA,2020,7022220.00,-2877.35,4623.62,49225.34,582476800000.00,31976.07,77021.00,82947.67,0.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
627,ND,2000,642023.00,-1746.20,2507.85,15879.49,18100100000.00,11551.14,25892.00,28192.29,0.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7031.00,6125.00
168,DE,1997,751487.00,-124.29,2100.24,20725.57,33353400000.00,21681.68,27069.00,44383.20,0.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8160.00,7420.00
809,OH,2014,11648446.00,-135.28,3471.64,27317.31,595640900000.00,20481.13,42829.00,51134.80,0.16,0.46,39.40,49308.00,38.10,907.00,81800.00,129100.00,198000.00,443.00,579.00,744.00,0.32,0.68,0.89,14214.00,11730.00
1077,VT,2018,640667.00,-146.41,5269.82,30170.43,33032700000.00,16265.86,52191.00,51559.86,0.10,0.45,43.10,60782.00,37.90,1184.00,157100.00,233100.00,340100.00,650.00,853.00,1170.00,0.25,0.75,0.78,20819.00,21217.00


### Import Education Level

In [358]:
edu_level = pd.read_csv('edu_level_state_annual_clean.csv')

edu_level.head(5)

,state,year,edu_associate_degree,edu_bachelor_degree,edu_high_school,edu_master_degree,edu_total
0,AL,2008,210308.00,439587.00,801711.00,170625.00,3078499.00
1,AL,2009,218551.00,447402.00,797598.00,168782.00,3115982.00
2,AL,2010,225382.00,439863.00,817024.00,182474.00,3161521.00
3,AL,2011,236162.00,443488.00,818373.00,190977.00,3193078.00
4,AL,2012,238555.00,472037.00,826740.00,193901.00,3209646.00


In [359]:
df_core = magic_merge(df_core, edu_level)

df_core.sample(10)

,state,year,population,subsidies,taxes_on_production_and_imports_(topi),gdp_compensation,gdp_current_dollar,gdp_gross_operating_surplus,per_capita_personal_income,per_cap_gdp_current_dollar,poverty_rate,gini_index,median_age,median_income,mean_usual_hours,median_monthly_house_cost,house_low_value_quartile,house_median_value,house_upper_value_quartile,rent_low_value_quartile,rent_median_value,rent_upper_value_quartile,household_renter_occ_ratio,household_owner_occ_ratio,households_occupied_ratio,per_pupil_revenue,per_pupil_expenditure,edu_associate_degree,edu_bachelor_degree,edu_high_school,edu_master_degree,edu_total
257,IA,2014,3121583.00,-328.97,3391.04,27013.95,173095900000.00,25375.30,44699.00,55451.32,0.10,0.44,38.20,53712.00,38.60,843.00,81900.00,133100.00,205600.00,421.00,570.00,752.00,0.26,0.74,0.91,12791.00,10938.00,233906.00,383943.00,571438.00,129844.00,2062734.00
802,OH,2007,11500468.00,-116.80,2982.17,24340.57,497761500000.00,16075.90,35629.00,43281.85,0.13,0.45,37.80,46597.00,38.70,975.00,93900.00,137800.00,199000.00,392.00,511.00,651.00,0.27,0.73,0.89,12476.00,10340.00,NaN,NaN,NaN,NaN,NaN
568,MS,2013,2986574.00,-151.18,2783.19,18399.24,102371000000.00,13245.85,34259.00,34277.07,0.19,0.48,36.60,37963.00,39.00,604.00,55400.00,97500.00,171300.00,363.00,529.00,715.00,0.33,0.67,0.85,8994.00,8265.00,162584.00,248488.00,478850.00,104166.00,1931054.00
599,MT,2020,1086193.00,-2309.35,3235.70,26940.61,51508800000.00,19554.44,53085.00,47421.41,0.11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1150,WV,2019,1800430.00,-74.20,3481.50,23417.07,79139700000.00,17131.69,42378.00,43956.00,0.14,0.46,42.90,48850.00,39.30,533.00,68700.00,124600.00,204400.00,394.00,551.00,717.00,0.25,0.75,0.81,NaN,NaN,101267.00,161686.00,431856.00,80380.00,1281086.00
587,MT,2008,976415.00,-413.25,2570.32,20049.06,37224900000.00,15917.92,35792.00,38124.06,0.13,0.45,39.10,43654.00,38.80,758.00,99000.00,180300.00,287000.00,372.00,515.00,689.00,0.29,0.71,0.86,11242.00,10120.00,49516.00,121576.00,167663.00,36162.00,648277.00
92,CA,2017,39424071.00,-189.34,4401.10,36851.66,2730973900000.00,28208.31,58813.00,69271.74,0.12,0.49,36.50,71805.00,38.20,1745.00,320500.00,509400.00,800200.00,925.00,1320.00,1881.00,0.44,0.56,0.92,14535.00,12664.00,2070661.00,5616449.00,4937316.00,2286009.00,26679273.00
1147,WV,2016,1835998.00,-77.56,3048.53,20362.22,70924400000.00,15296.69,37380.00,38629.89,0.18,0.47,42.30,43385.00,38.50,527.00,65900.00,117900.00,190100.00,354.00,521.00,686.00,0.26,0.74,0.81,12877.00,11745.00,95794.00,162243.00,422721.00,78802.00,1291757.00
566,MS,2011,2977897.00,-228.52,2588.74,17467.33,96218000000.00,12483.17,32546.00,32310.72,0.17,0.47,36.10,36919.00,39.40,643.00,NaN,NaN,NaN,NaN,NaN,NaN,0.30,0.70,0.84,9052.00,8097.00,159186.00,240726.00,471391.00,95403.00,1912708.00
56,AZ,2005,5839077.00,-119.80,2744.54,20855.68,227915900000.00,15552.44,32327.00,39032.86,0.15,NaN,34.50,44282.00,39.90,952.00,116000.00,185400.00,303500.00,486.00,621.00,806.00,0.30,0.70,0.87,8071.00,6515.00,NaN,NaN,NaN,NaN,NaN


In [360]:
# convert edu level to percent of population
df_core[['edu_associate_degree','edu_bachelor_degree','edu_high_school','edu_master_degree']] = df_core[['edu_associate_degree','edu_bachelor_degree','edu_high_school','edu_master_degree']].div(df_core['edu_total'], axis=0)

df_core.sample(5)

,state,year,population,subsidies,taxes_on_production_and_imports_(topi),gdp_compensation,gdp_current_dollar,gdp_gross_operating_surplus,per_capita_personal_income,per_cap_gdp_current_dollar,poverty_rate,gini_index,median_age,median_income,mean_usual_hours,median_monthly_house_cost,house_low_value_quartile,house_median_value,house_upper_value_quartile,rent_low_value_quartile,rent_median_value,rent_upper_value_quartile,household_renter_occ_ratio,household_owner_occ_ratio,households_occupied_ratio,per_pupil_revenue,per_pupil_expenditure,edu_associate_degree,edu_bachelor_degree,edu_high_school,edu_master_degree,edu_total
1164,WY,2009,559851.00,-145.40,6425.10,27253.14,36252000000.00,31219.91,44564.00,64752.94,0.09,0.41,36.70,52664.00,39.80,869.00,113700.00,184000.00,268000.00,430.00,583.00,780.00,0.26,0.74,0.86,19497.00,15232.00,0.09,0.16,0.25,0.06,355793.00
1082,WA,1999,5842564.00,-149.39,3363.11,21510.95,229797900000.00,14607.01,30958.00,39331.69,0.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7546.00,6376.00,NaN,NaN,NaN,NaN,NaN
172,DE,2001,795699.00,-140.38,2188.14,25137.02,46096100000.00,30746.93,36922.00,57931.58,0.07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9841.00,9284.00,NaN,NaN,NaN,NaN,NaN
274,ID,2007,1505105.00,-139.92,2306.55,19924.86,53934500000.00,13742.90,32707.00,35834.38,0.10,0.44,34.30,46253.00,39.20,909.00,119800.00,178100.00,278000.00,415.00,564.00,729.00,0.26,0.74,0.89,7965.00,6951.00,NaN,NaN,NaN,NaN,NaN
238,GA,2019,10642947.00,-163.89,3594.01,31045.62,637798700000.00,25451.15,48697.00,59926.89,0.12,0.48,37.20,61980.00,39.40,1059.00,124500.00,202500.00,327500.00,580.00,857.00,1190.00,0.34,0.66,0.88,NaN,NaN,0.08,0.20,0.23,0.09,7080222.00


### Import PCE Values

In [361]:
pce = pd.read_csv('pce_function_state_annual_clean.csv')

pce.head(5)

,year,state,pce_clothing_footwear_and_related_services,pce_communication,pce_education,pce_financial_services_and_insurance,pce_food_and_beverages_purchased_for_off-premises_consumption,pce_food_services_and_accommodations,pce_furnishings_household_equipment_and_routine_household_maintenance,pce_health,pce_housing_utilities_and_fuels,pce_other_goods_and_services,pce_total,pce_recreation,pce_transportation
0,1997,AK,552000000.00,193700000.00,255200000.00,848300000.00,1450000000.00,1091000000.00,701300000.00,1965500000.00,2457500000.00,1146800000.00,14205600000.00,1451200000.00,1711600000.00
1,1997,AL,3457300000.00,1875300000.00,1156500000.00,5026600000.00,7747300000.00,4278700000.00,3993800000.00,14574600000.00,13361000000.00,4375700000.00,77070900000.00,6744600000.00,9454500000.00
2,1997,AR,2186700000.00,881100000.00,715600000.00,2953100000.00,3952400000.00,2370000000.00,2245300000.00,7505800000.00,7595600000.00,2464000000.00,41941200000.00,3290300000.00,5312800000.00
3,1997,AZ,3830400000.00,1930400000.00,1600000000.00,6674700000.00,8135900000.00,6703500000.00,5342900000.00,13544900000.00,20212800000.00,6564900000.00,96365600000.00,8586500000.00,12454400000.00
4,1997,CA,33257500000.00,15812800000.00,12700500000.00,51494500000.00,54740400000.00,42657600000.00,32307800000.00,106739900000.00,132581500000.00,42047000000.00,676652200000.00,69924000000.00,75773900000.00


In [362]:
# Get State population df to create per_cap_pce
pop = df_core[['state','year','population']]

pce = pd.merge(pce, pop, how='inner', on=['state','year'])

In [363]:
pce.head(2)

,year,state,pce_clothing_footwear_and_related_services,pce_communication,pce_education,pce_financial_services_and_insurance,pce_food_and_beverages_purchased_for_off-premises_consumption,pce_food_services_and_accommodations,pce_furnishings_household_equipment_and_routine_household_maintenance,pce_health,pce_housing_utilities_and_fuels,pce_other_goods_and_services,pce_total,pce_recreation,pce_transportation,population
0,1997,AL,3457300000.00,1875300000.00,1156500000.00,5026600000.00,7747300000.00,4278700000.00,3993800000.00,14574600000.00,13361000000.00,4375700000.00,77070900000.00,6744600000.00,9454500000.00,4367935.00
1,1997,AR,2186700000.00,881100000.00,715600000.00,2953100000.00,3952400000.00,2370000000.00,2245300000.00,7505800000.00,7595600000.00,2464000000.00,41941200000.00,3290300000.00,5312800000.00,2601090.00


In [364]:
pce.iloc[:, 2:15]

,pce_clothing_footwear_and_related_services,pce_communication,pce_education,pce_financial_services_and_insurance,pce_food_and_beverages_purchased_for_off-premises_consumption,pce_food_services_and_accommodations,pce_furnishings_household_equipment_and_routine_household_maintenance,pce_health,pce_housing_utilities_and_fuels,pce_other_goods_and_services,pce_total,pce_recreation,pce_transportation
0,3457300000.00,1875300000.00,1156500000.00,5026600000.00,7747300000.00,4278700000.00,3993800000.00,14574600000.00,13361000000.00,4375700000.00,77070900000.00,6744600000.00,9454500000.00
1,2186700000.00,881100000.00,715600000.00,2953100000.00,3952400000.00,2370000000.00,2245300000.00,7505800000.00,7595600000.00,2464000000.00,41941200000.00,3290300000.00,5312800000.00
2,3830400000.00,1930400000.00,1600000000.00,6674700000.00,8135900000.00,6703500000.00,5342900000.00,13544900000.00,20212800000.00,6564900000.00,96365600000.00,8586500000.00,12454400000.00
3,33257500000.00,15812800000.00,12700500000.00,51494500000.00,54740400000.00,42657600000.00,32307800000.00,106739900000.00,132581500000.00,42047000000.00,676652200000.00,69924000000.00,75773900000.00
4,4048200000.00,1340800000.00,1730600000.00,6674600000.00,7265300000.00,6756600000.00,4964200000.00,12165600000.00,17467200000.00,5908900000.00,89413400000.00,10576500000.00,10965900000.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1171,522800000.00,313600000.00,745400000.00,1809800000.00,2755200000.00,1461400000.00,1200800000.00,6560400000.00,5985700000.00,1593500000.00,29544800000.00,2208600000.00,2439600000.00
1172,8491600000.00,11088800000.00,5819800000.00,26071200000.00,29515900000.00,18046400000.00,15635900000.00,68523400000.00,73310800000.00,21605400000.00,354219100000.00,37076600000.00,27244900000.00
1173,5578300000.00,3188900000.00,4591200000.00,19467500000.00,18785500000.00,12066800000.00,10287100000.00,54645300000.00,42732100000.00,12915500000.00,238923000000.00,20975500000.00,21598800000.00
1174,1448300000.00,847600000.00,999600000.00,4565000000.00,5929100000.00,3348900000.00,2593800000.00,18753100000.00,11210500000.00,3573500000.00,66404200000.00,3900900000.00,5381500000.00


In [365]:
# divide PCE values to get per capita PCE
pce.iloc[:, 2:15] = pce.iloc[:, 2:15].div(pce['population'], axis=0)

# drop population post math operation
pce = pce.drop(columns=['population'])

pce.sample(10)

,year,state,pce_clothing_footwear_and_related_services,pce_communication,pce_education,pce_financial_services_and_insurance,pce_food_and_beverages_purchased_for_off-premises_consumption,pce_food_services_and_accommodations,pce_furnishings_household_equipment_and_routine_household_maintenance,pce_health,pce_housing_utilities_and_fuels,pce_other_goods_and_services,pce_total,pce_recreation,pce_transportation
605,2009,MA,1357.57,840.72,1086.13,3510.18,2949.59,2481.22,1531.45,9089.34,7257.90,2737.49,41106.83,3685.52,3237.18
1005,2017,NC,1095.88,764.19,722.33,2739.04,2857.41,2510.72,1682.84,7979.27,6544.78,2489.88,36133.38,3145.08,3489.30
511,2007,MN,1103.11,539.52,838.88,2843.45,2352.22,1956.54,1471.54,6895.58,6308.11,2278.21,34935.76,3405.80,3547.05
456,2006,KY,877.36,383.79,560.39,1647.07,2353.39,1606.85,1203.32,5719.06,4417.79,1716.16,26454.27,2009.51,3143.34
755,2012,MI,975.57,546.45,835.74,2906.03,2601.81,1877.64,1223.75,7062.76,6226.29,1981.37,33467.09,2419.23,3045.26
1068,2018,SD,1081.23,751.63,651.89,3357.48,2816.86,2664.46,1673.71,11180.05,6409.69,2133.44,41379.30,3232.15,3545.64
59,1998,IA,808.75,391.51,433.67,1094.40,1644.06,1113.52,912.34,3448.89,3375.28,1287.38,19555.46,1954.44,2451.68
150,2000,CA,1154.24,635.51,458.31,2028.89,1875.07,1519.40,1247.48,3619.97,4638.92,1593.70,24660.90,2679.53,2860.86
524,2007,OK,987.44,619.07,527.71,1844.04,1884.46,1552.11,1282.46,5569.17,4762.09,1703.03,26931.84,2235.78,3602.65
410,2005,MD,1275.32,732.21,836.16,2455.40,2299.86,1875.39,1686.58,5975.04,6334.91,2345.37,33142.94,3143.83,3388.59


In [366]:
df_core = magic_merge(df_core, pce)

df_core.head(10)

,state,year,population,subsidies,taxes_on_production_and_imports_(topi),gdp_compensation,gdp_current_dollar,gdp_gross_operating_surplus,per_capita_personal_income,per_cap_gdp_current_dollar,poverty_rate,gini_index,median_age,median_income,mean_usual_hours,median_monthly_house_cost,house_low_value_quartile,house_median_value,house_upper_value_quartile,rent_low_value_quartile,rent_median_value,rent_upper_value_quartile,household_renter_occ_ratio,household_owner_occ_ratio,households_occupied_ratio,per_pupil_revenue,per_pupil_expenditure,edu_associate_degree,edu_bachelor_degree,edu_high_school,edu_master_degree,edu_total,pce_clothing_footwear_and_related_services,pce_communication,pce_education,pce_financial_services_and_insurance,pce_food_and_beverages_purchased_for_off-premises_consumption,pce_food_services_and_accommodations,pce_furnishings_household_equipment_and_routine_household_maintenance,pce_health,pce_housing_utilities_and_fuels,pce_other_goods_and_services,pce_total,pce_recreation,pce_transportation
0,AL,1997,4367935.00,-62.84,1546.45,13984.59,104811900000.00,8527.58,21516.00,23995.76,0.16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5535.00,4849.00,NaN,NaN,NaN,NaN,NaN,791.52,429.33,264.77,1150.80,1773.68,979.57,914.35,3336.73,3058.88,1001.78,17644.70,1544.12,2164.52
1,AL,1998,4404701.00,-66.50,1582.01,14568.21,110212000000.00,8937.75,22668.00,25021.45,0.14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5975.00,5188.00,NaN,NaN,NaN,NaN,NaN,825.60,442.10,282.47,1225.83,1785.91,1013.60,942.74,3452.36,3179.88,1044.50,18271.61,1592.50,2204.33
2,AL,1999,4430141.00,-87.67,1654.48,15174.48,115680100000.00,9370.74,23333.00,26112.06,0.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6523.00,5638.00,NaN,NaN,NaN,NaN,NaN,858.71,460.96,300.35,1312.78,1843.37,1064.26,977.24,3561.44,3319.65,1124.68,19194.47,1667.71,2385.66
3,AL,2000,4452173.00,-81.92,1767.36,15669.74,119851700000.00,9564.63,24306.00,26919.82,0.13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6503.00,5885.00,NaN,NaN,NaN,NaN,NaN,878.27,485.99,324.31,1445.23,1872.30,1094.95,1001.35,3759.89,3531.02,1173.99,20198.90,1722.22,2551.47
4,AL,2001,4467634.00,-106.50,1791.62,16124.51,122915500000.00,9702.81,25057.00,27512.44,0.16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6956.00,6029.00,NaN,NaN,NaN,NaN,NaN,875.12,492.95,334.29,1423.42,1968.40,1111.80,1002.34,4053.55,3771.53,1220.27,20911.81,1738.33,2543.76
5,AL,2002,4480089.00,-112.65,1796.88,16551.52,127505000000.00,10224.60,25648.00,28460.37,0.14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6971.00,6300.00,NaN,NaN,NaN,NaN,NaN,877.42,501.06,344.50,1463.01,2018.82,1144.82,1040.87,4432.93,3883.81,1290.71,21744.66,1805.37,2559.68
6,AL,2003,4503491.00,-103.85,1872.92,17116.72,134152600000.00,10902.79,26673.00,29788.58,0.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7349.00,6581.00,NaN,NaN,NaN,NaN,NaN,884.58,500.10,361.72,1563.03,2073.55,1220.05,1082.34,4681.37,4005.60,1363.16,22718.64,1931.92,2686.78
7,AL,2004,4530729.00,-91.91,1985.49,18007.90,147715200000.00,12701.49,28434.00,32602.97,0.17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8028.00,7073.00,NaN,NaN,NaN,NaN,NaN,922.21,509.39,374.99,1635.92,2137.96,1288.95,1146.39,4979.62,4200.30,1421.85,23980.03,2093.04,2867.44
8,AL,2005,4569805.00,-122.65,2169.35,18897.26,158846800000.00,13816.13,29949.00,34760.08,0.17,NaN,37.40,36879.00,39.80,624.00,59400.00,97500.00,162500.00,272.00,394.00,541.00,0.28,0.72,0.86,8555.00,7683.00,NaN,NaN,NaN,NaN,NaN,962.21,503.54,393.06,1743.60,2195.80,1374.76,1221.26,5300.25,4468.79,1476.91,25355.26,2215.46,3079.61
9,AL,2006,4628981.00,-108.77,2283.72,19690.73,166469000000.00,14096.69,31474.00,35962.34,0.14,0.47,37.20,38783.00,39.80,685.00,64200.00,107000.00,174900.00,282.00,410.00,562.00,0.26,0.74,0.85,9548.00,8398.00,NaN,NaN,NaN,NaN,NaN,984.67,510.31,407.82,1796.33,2216.25,1448.98,1259.46,5523.59,4705.12,1517.59,26312.25,2316.90,3196.73


### Import Business Applications data

In [367]:
busi_apps = pd.read_csv('business_apps_state_annual_clean.csv')

busi_apps.head(5)

,sa,geo,year,naics_sector,business_applications,business_applications_from_corporations,high‐propensity_business_applications_ba_wba___business_applications_with_planned_wages
0,A,AK,2005,Total for All NAICS,4770.00,1028.00,2708.00
1,A,AK,2006,Total for All NAICS,5218.00,1142.00,2961.00
2,A,AK,2007,Total for All NAICS,5197.00,1024.00,2855.00
3,A,AK,2008,Total for All NAICS,4902.00,749.00,2428.00
4,A,AK,2009,Total for All NAICS,4452.00,607.00,2122.00


In [368]:
busi_apps = busi_apps.rename(columns={'high‐propensity_business_applications_ba_wba___business_applications_with_planned_wages':'high_propensity_business_application',
                                      'geo':'state'})

In [369]:
df_core = magic_merge(df_core, busi_apps)

df_core.sample(10)

,state,year,population,subsidies,taxes_on_production_and_imports_(topi),gdp_compensation,gdp_current_dollar,gdp_gross_operating_surplus,per_capita_personal_income,per_cap_gdp_current_dollar,poverty_rate,gini_index,median_age,median_income,mean_usual_hours,median_monthly_house_cost,house_low_value_quartile,house_median_value,house_upper_value_quartile,rent_low_value_quartile,rent_median_value,rent_upper_value_quartile,household_renter_occ_ratio,household_owner_occ_ratio,households_occupied_ratio,per_pupil_revenue,per_pupil_expenditure,edu_associate_degree,edu_bachelor_degree,edu_high_school,edu_master_degree,edu_total,pce_clothing_footwear_and_related_services,pce_communication,pce_education,pce_financial_services_and_insurance,pce_food_and_beverages_purchased_for_off-premises_consumption,pce_food_services_and_accommodations,pce_furnishings_household_equipment_and_routine_household_maintenance,pce_health,pce_housing_utilities_and_fuels,pce_other_goods_and_services,pce_total,pce_recreation,pce_transportation,sa,naics_sector,business_applications,business_applications_from_corporations,high_propensity_business_application
1076,VT,2017,639061.00,-136.45,5075.88,29282.81,32246900000.00,16237.57,50709.00,50459.82,0.09,0.45,42.60,57513.00,38.00,1176.00,153600.00,226300.00,335300.00,623.00,844.00,1160.00,0.27,0.73,0.77,20212.00,20149.00,0.09,0.22,0.26,0.11,439267.00,900.85,507.93,1139.48,2817.41,3693.54,3088.28,1644.44,9814.56,8330.97,2520.10,44974.58,3618.59,4194.44,A,Total for All NAICS,5416.00,428.00,1781.00
216,GA,1997,7685099.00,-90.02,2105.76,17210.58,242540800000.00,12333.56,24404.00,31559.88,0.14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6571.00,5647.00,NaN,NaN,NaN,NaN,NaN,1026.08,336.09,300.95,1448.92,1726.54,1341.43,1104.68,3207.53,3691.46,1260.07,19750.34,1727.41,2604.78,NaN,NaN,NaN,NaN,NaN
149,DC,2002,573158.00,-852.47,3540.73,85925.35,69919500000.00,33376.49,45147.00,121989.92,0.17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14626.00,11847.00,NaN,NaN,NaN,NaN,NaN,1630.61,1020.31,915.28,3026.91,3183.59,2657.21,1710.00,7199.06,5695.64,6280.64,42114.74,3256.00,4279.97,NaN,NaN,NaN,NaN,NaN
1149,WV,2018,1810588.00,-70.03,3456.17,23214.67,79044800000.00,17056.12,41300.00,43656.98,0.16,0.47,42.80,44097.00,39.20,552.00,70400.00,121300.00,198100.00,393.00,546.00,704.00,0.26,0.74,0.82,13503.00,12269.00,0.08,0.13,0.33,0.06,1280781.00,848.84,465.54,563.02,2572.09,2903.09,2209.72,1289.30,10231.70,6103.65,2051.38,36775.07,2216.19,3401.44,A,Total for All NAICS,11185.00,871.00,3546.00
260,IA,2017,3162621.00,-372.73,3773.42,29318.72,183549800000.00,25317.83,47286.00,58037.24,0.07,0.44,38.30,58570.00,39.00,879.00,90300.00,149100.00,232400.00,465.00,628.00,808.00,0.25,0.75,0.90,13772.00,11724.00,0.12,0.19,0.27,0.07,2093909.00,948.45,557.51,689.62,2562.21,2845.49,2186.26,1329.12,7912.14,6292.47,2230.81,36432.85,3084.85,4311.99,A,Total for All NAICS,22752.00,2370.00,7711.00
786,NY,2015,20088855.00,-320.33,5885.29,38615.74,1487627600000.00,29871.68,57481.00,74052.38,0.14,0.51,38.30,60850.00,38.50,1381.00,140700.00,293500.00,502700.00,666.00,1058.00,1544.00,0.44,0.56,0.88,24717.00,22231.00,0.08,0.20,0.23,0.11,13641473.00,1487.18,961.23,918.83,3889.81,2723.36,3019.09,1564.05,9344.94,7695.70,3242.80,42776.92,3204.18,3209.21,A,Total for All NAICS,201028.00,76468.00,105126.00
153,DC,2006,570681.00,-1152.31,5840.39,107753.72,88125000000.00,41978.97,55805.00,154420.77,0.18,0.54,35.00,51847.00,40.30,1558.00,299000.00,437700.00,701700.00,541.00,822.00,1192.00,0.52,0.48,0.89,17602.00,15511.00,NaN,NaN,NaN,NaN,NaN,1949.07,1287.93,1102.54,4096.33,3946.51,3776.37,2174.95,9005.38,7016.88,7969.78,52795.17,4028.70,5311.72,A,Total for All NAICS,7997.00,1740.00,3818.00
1048,VA,2013,8269638.00,-141.71,3568.72,31347.74,457640500000.00,20565.10,48573.00,55339.85,0.10,0.47,37.60,62666.00,39.00,1233.00,145700.00,239300.00,394800.00,614.00,922.00,1406.00,0.33,0.67,0.90,11814.00,10955.00,0.07,0.21,0.21,0.11,5561224.00,1223.63,914.38,922.47,2544.

In [370]:
df_core.shape

(1176, 50)

In [371]:
df_core.to_csv('../core_df.csv', index=False)